In [1]:
from psycopg2 import connect
import configparser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# DB Connection
CONFIG = configparser.ConfigParser(interpolation=None)
CONFIG.read('./ttc_db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [3]:
sql = '''
DROP MATERIALIZED VIEW IF EXISTS headway_data CASCADE; 
CREATE MATERIALIZED VIEW headway_data AS 
WITH ordered_trains AS (
  SELECT subwayline, station_char, trainid, timint, requestid,
  RANK() OVER (PARTITION BY subwayline, station_char, requestid ORDER BY timint, trainid) as train_order
  FROM ntas_data
), closest_trains AS (
  SELECT subwayline, station_char, trainid, timint, requestid
  FROM ordered_trains
  WHERE train_order = 1
), valid_trains_next AS (
  SELECT subwayline, station_char, trainid, timint, requestid, request_date,
    DATE(request_date - INTERVAL '3 hours') as adj_date,
    LEAD(trainid) OVER (PARTITION BY subwayline, station_char, DATE(request_date - INTERVAL '3 hours') ORDER BY request_date, requestid) as next_train
  FROM closest_trains
  JOIN requests USING (requestid)
  WHERE (
  (request_date::time <= '01:30:00'::time
   OR extract(dow from request_date) > 0 AND extract(hour from request_date) >= 6)
   OR (extract(dow from request_date) = 0 AND extract(hour from request_date) >= 8)
   )
)
  SELECT subwayline, station_char, trainid, timint, requestid, adj_date, request_date, 
  LEAD(request_date) OVER (PARTITION BY subwayline, station_char, DATE(request_date - INTERVAL '3 hours') ORDER BY request_date, requestid) AS next_train_request_date,
  LEAD(timint) OVER (PARTITION BY subwayline, station_char, DATE(request_date - INTERVAL '3 hours') ORDER BY request_date, requestid) AS next_train_timint
  FROM valid_trains_next
  WHERE trainid <> next_train
''' 
with con:
    with con.cursor() as cur:
        cur.execute(sql)

In [4]:
pd.set_option('float_format', '{0:.2f}'.format)

sql = '''
SELECT station_char, EXTRACT(EPOCH FROM next_train_request_date-request_date) / 60 as headway
FROM headway_data
WHERE subwayline = 'BD'
AND RIGHT(station_char, 1) = '1'
'''
data = pd.read_sql(sql, con)
data.groupby('station_char').describe()


headway                                     
               count mean  std  min  25%  50%  75%    max
station_char                                             
BAT1         7550.00 3.84 2.51 0.60 2.13 3.72 5.00 136.74
BAU1         7555.00 3.81 2.53 0.60 2.12 3.39 5.00 140.00
BLO1            0.00  nan  nan  nan  nan  nan  nan    nan
BRD1         7658.00 3.74 2.54 0.60 2.00 3.13 5.00 139.00
BSP1         7943.00 3.65 2.33 0.00 2.13 3.27 4.88 137.74
CFK1         7673.00 3.74 2.50 0.60 2.01 3.26 5.00 138.01
CHE1         8035.00 3.56 2.64 0.25 2.00 3.00 4.98 140.00
CHR1         7531.00 3.85 2.54 0.59 2.13 3.73 5.00 140.14
COX1         8302.00 3.50 2.51 0.60 2.00 3.00 4.87 137.74
DNW1         7743.00 3.75 2.43 0.60 2.25 3.40 4.99 137.74
DON1         7999.00 3.57 2.50 0.60 2.00 3.00 4.89 136.00
DUF1         7796.00 3.72 2.36 0.58 2.13 3.39 4.99 137.74
GWD1         8561.00 3.39 2.46 0.60 2.00 3.00 4.74 135.27
HPK1         7456.00 3.89 2.57 0.60 2.38 3.74 5.00 136.00
ISL1         7150.00 4.05 2.79 0.59 2.74 3.87 5.00 136.00
JNE1         7420.00 3.92 2.59 0.58 2.40 3.85 5.00 137.74
KEL1         7578.00 3.83 2.48 0.60 2.27 3.66 5.00 136.74
KEN1         5471.00 5.30 3.51 0.60 3.13 5.00 7.00 142.99
KIP1         4720.00 6.13 3.67 0.60 4.00 6.00 8.13 137.74
LAN1         7804.00 3.72 2.37 0.59 2.13 3.40 4.88 136.74
MST1         8136.00 3.57 2.54 0.60 2.00 3.00 4.98 137.87
OML1         7418.00 3.91 2.54 0.59 2.40 3.75 5.00 136.74
OSS1         7606.00 3.81 2.46 0.60 2.13 3.74 5.00 139.00
PAP1         8037.00 3.56 2.52 0.08 2.00 3.00 4.88 136.61
RUN1         7453.00 3.90 2.57 0.59 2.27 3.85 5.00 139.00
RYK1         7397.00 3.92 2.57 0.59 2.60 3.87 5.00 137.74
SGL1         7948.00 3.65 2.35 0.00 2.12 3.27 4.88 139.00
SGU1            0.00  nan  nan  nan  nan  nan  nan    nan
SHE1         7626.00 3.76 2.53 0.60 2.01 3.27 5.00 137.74
VPK1         8094.00 3.58 2.54 0.60 2.00 3.00 4.89 135.27
WAR1         8148.00 3.56 2.54 0.60 2.00 3.00 4.87 137.00
WDB1         8140.00 3.57 2.53 0.61 2.00 3.00 4.88 137.00
YNG1         7811.00 3.68 2.41 0.00 2.02 3.13 4.98 135.87